Load some utils function + annotations first

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
from utils import load_image_annotations, display_image_with_keypoints

In [ ]:
annotations = json.load(open("/root/data/gtsf_2.0/registration_test/gtsf_in_air_labels.json"))

load reference image

In [ ]:
static_image, static_mask, static_keypoints, static_kp_map = load_image_annotations(annotations[0], new_shape=(800, 600))

In [ ]:
display_image_with_keypoints(static_image, static_keypoints)

load one moving image

In [ ]:
moving_image, moving_mask, moving_keypoints, moving_kp_map = load_image_annotations(annotations[5], new_shape=(800, 600))

In [ ]:
display_image_with_keypoints(moving_image, moving_keypoints)

some dipy stuff

In [ ]:
from copy import copy

from dipy.viz import regtools

from utils import translate_moving

In [ ]:
regtools.overlay_images(static_mask, moving_mask, 'Static', 'Overlay', 'Moving', 'input_images.png')

In [ ]:
moving_translated, translation_vector = translate_moving(static_mask, moving_mask)

In [ ]:
regtools.overlay_images(static_mask, moving_translated, 'Static', 'Overlay', 'Moving', 'input_images.png')

Perform the registration

In [ ]:
from utils import register

In [ ]:
mapping = register(static_mask, moving_translated)

In [ ]:
regtools.plot_2d_diffeomorphic_map(mapping, 10, 'diffeomorphic_map.png')

Display forward registration

In [ ]:
warped_moving = mapping.transform(moving_translated, 'linear')
regtools.overlay_images(static_mask, warped_moving, 'Static','Overlay','Warped moving',
   'direct_warp_result.png')

Display backward registration

In [ ]:
warped_static = mapping.transform_inverse(static_mask, 'linear')
regtools.overlay_images(warped_static, moving_translated,'Warped static','Overlay','Moving',
   'inverse_warp_result.png')

Warping keypoints

In [ ]:
from utils import display_warped_keypoints

In [ ]:
warped_kp_map = mapping.transform_inverse(static_kp_map, "linear")

In [ ]:
display_warped_keypoints(moving_image, warped_kp_map, translation_vector)

In [ ]:
from utils import display_pairs_with_keypoints

In [ ]:
display_pairs_with_keypoints(moving_image, moving_kp_map, warped_kp_map, translation_vector)

# TEST

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from scipy.optimize import linear_sum_assignment

In [ ]:
from utils import calculate_errors

In [ ]:
calculate_errors(moving_keypoints, warped_kp_map, translation_vector)

In [ ]:
distances = pairwise_distances(moving_keypoints.transpose())

In [ ]:
keypoints_mv = []
same = []
for i in range(distances.shape[0]):
    if i in list(same):
        continue
    same = np.where(distances[i, :] < 10)[0]
    x = int(np.mean([moving_keypoints[0, k] for k in same]))
    y = int(np.mean([moving_keypoints[1, k] for k in same]))
    keypoints_mv.append((x,y))

In [ ]:
distances = pairwise_distances(warped_keypoints.transpose())

In [ ]:
keypoints_wrp = []
same = []
for i in range(distances.shape[0]):
    if i in list(same):
        continue
    same = np.where(distances[i, :] < 10)[0]
    print(same)
    x = int(np.mean([warped_keypoints[0, k] for k in same]))
    y = int(np.mean([warped_keypoints[1, k] for k in same]))
    keypoints_wrp.append((x,y))

In [ ]:
keypoints_mv

In [ ]:
keypoints_wrp